## Were steady states caused by initial condition bias towards 1?

Dynamical system in the thinning paper
Node saturation
Spectral design
Topology paper

In [1]:
%run "../HyperParameterOpt/GenerateExperiments/res_experiment.py"
import copy
from matplotlib import pyplot as plt
import scipy as sp
from scipy import signal
DIFF_EQ_PARAMS = {
                  "x0": [-20, 10, -.5],
                  "begin": 0,
                  "end": 85,
                  "timesteps": 85000,
                  "train_per": .889,
                  "solver": lorenz_equ,
                  "clip": 40
                 }

RES_PARAMS = {
              "uniform_weights": True,
              "solver": "ridge",
              "ridge_alpha": 1e-6,
              "signal_dim": 3,
              "network": "random graph",

              "res_sz": 2500,
              "activ_f": np.tanh,
              "connect_p": .1,
              "spect_rad": 0.5, #5,
              "gamma": 5,
              "sigma": .14,
              "sparse_res": True,
             }

TOL = 5
plt.rcParams["figure.figsize"] = [10, 5]

In [23]:
def adj_matrix(topology, topo_params, n, remove_p=0):
    adj = generate_adj(topology, topo_params, n)
    adj = remove_edges(adj, floor(remove_p*np.sum(adj != 0)))
    return adj

def steady_state_ratio(topo, topo_p, remove_p=0.0):
    DIFF_EQ_PARAMS["x0"] = random_lorenz_x0()
    train_t, test_t, u = rc_solve_ode(DIFF_EQ_PARAMS)
    A = adj_matrix(topo, topo_p, 2500, remove_p=remove_p)
    rc = ResComp(A, **RES_PARAMS)
    rc.state_0 = 2*np.random.rand(rc.res_sz) - 1
    r0 = rc.state_0
    states = rc.drive(train_t, u)
    rank = np.linalg.matrix_rank(states)
    mask = np.abs(np.mean(states, axis=0)) > .97
    rc.state_0 = r0
    error = rc.fit(train_t, u) # Some special kind of L2 error
    mean_ss = np.mean(np.abs(rc.W_out[:, mask]))
    mean_os = np.mean(np.abs(rc.W_out[:, ~mask]))
    n = sum(mask)
    return np.round(mean_ss, 4), np.round(mean_os,4), n, error, rank

def rep_ss_ratio(topo, topo_p=None, remove_p=0.0, N=10):
    totss = 0
    totss_weight = 0
    totos_weight = 0
    toterr = 0
    totrank = 0
    for i in range(N):
        ss, os, n, error, rank = steady_state_ratio(topo, topo_p, remove_p=remove_p)
        totss += n
        totss_weight+= ss
        totos_weight += os
        toterr += error
        totrank += rank
        print(f"Trial {i}: {n} Steady states. Avg weight={ss} Oscil. w avg={os} Ratio={np.round(ss/os, 4)} Error={np.round(error, 4)}, Rank={rank}")
    perss = np.round(totss/(N*RES_PARAMS["res_sz"]), 4)
    avgss_weight = np.round(totss_weight/N, 4)
    avgos_weight = np.round(totss/N, 4)
    ratio = np.round(totss_weight/totos_weight, 4)
    avgerr = np.round(toterr/N, 6)
    avgrank = np.round(totrank/N, 4)
    print(f"Average: %ss={perss} ss_weight={avgss_weight} os_weight={avgos_weight} ratio={ratio}, err={avgerr}, rank={avgrank}")

## Watts (Remove 0%)

In [24]:
rep_ss_ratio("watts4", topo_p=0.1, remove_p=0.0, N=1)

Trial 0: 1815 Steady states. Avg weight=1.7341 Oscil. w avg=8.7661 Ratio=0.1978 Error=0.0064, Rank=1085
Average: %ss=0.726 ss_weight=1.7341 os_weight=1815.0 ratio=0.1978, err=0.006436, rank=1085.0


## Watts (Remove 90%)

In [25]:
rep_ss_ratio("watts4", topo_p=0.1, remove_p=0.9, N=1)

Trial 0: 433 Steady states. Avg weight=3.0454 Oscil. w avg=3.579 Ratio=0.8509 Error=0.0054, Rank=906
Average: %ss=0.1732 ss_weight=3.0454 os_weight=433.0 ratio=0.8509, err=0.005449, rank=906.0


## Geometric (Remove 0%)

In [ ]:
rep_ss_ratio("geom", topo_p=4, remove_p=0.0)

## Geometric (Remove 90%)

In [ ]:
rep_ss_ratio("geom", topo_p=4.0, remove_p=0.9)

## Identity (Remove 0%)

In [ ]:
rep_ss_ratio("ident", topo_p=None, remove_p=0.0)

## Identity (Remove 90%)

In [ ]:
rep_ss_ratio("ident", topo_p=None, remove_p=0.9)

## Barabasi (Remove 0%)

In [ ]:
rep_ss_ratio("barab2", topo_p=None, remove_p=0.0)

## Barabasi (Remove 90%)

In [ ]:
rep_ss_ratio("barab2", topo_p=None, remove_p=0.9)

## Random Digraph (Remove 0%)

In [ ]:
rep_ss_ratio("random_digraph", topo_p=4.0, remove_p=0.0)

## Random Digraph (Remove 90%)

In [ ]:
rep_ss_ratio("random_digraph", topo_p=4.0, remove_p=0.9)

## Erdos (Remove 0%)

In [ ]:
rep_ss_ratio("erdos", topo_p=4.0, remove_p=0.0)

## Erdos (Remove 90%)

In [ ]:
rep_ss_ratio("erdos", topo_p=4.0, remove_p=0.9)